In [1]:
from IPython.display import Image
Image(url='https://vesg.ipsl.upmc.fr/thredds/fileServer/IPSLFS/jservon/CliMAF_Notebooks_html/CliMAF-logo-small.png') 

### A science-oriented framework to ease the analysis of climate model simulations
##### WP5 ANR Convergence
##### Project leaders: Stéphane Sénési (CNRM-GAME), Jérôme Servonnat (LSCE-IPSL)
##### Developers: Ludivine Vignon (CNRM-GAME), Laurent Franchisteguy (CNRM-GAME), Patrick Brockmann (LSCE-IPSL), Sylvana Ramos-Buarque (CNRM-GAME)
##### Beta-testing: Olivier Marti (LSCE-IPSL), Marie-Pierre Moine (CERFACS), Emilia Sanchez-Gomez (CERFACS)
##### Contributors: you!
##   
##### contact: climaf@meteo.fr

The goals of CliMAF are to provide the scientists with simplified and science-oriented means for :
- accessing both model and references data
- pre-tretament (period and geographical selections, regridding, averaging like seasonal mean computations...)
- plotting  maps, cross-sections and time series
- building  atlases
- plugging personal scripts in an atlas or in an analysis workflow
- sharing such scripts
- handlling ensembles (multi-model, multi-realization)
CliMAF provides full managment of the outputs by handling the naming of the output files; it also stores the information on "how I obtained this file" under the form of an expression called CRS (CliMAF Reference Syntax) that allows checking for the existing files and avoid recomputing the same files twice (or more).

#  
#   

# Understand how the CliMAF cache works and manage the content

In this notebook we will see how the CliMAF cache works and how to:
- 1. totally erase the content of the cache
- 2. CliMAF stores its results and knows that those results exist (and are linked with sequences of operations)
- 3. remove one specific result
- 4. remove results using criteria: pattern, age...
- 5. remove everything but a list of results you want to keep
- 6. csync: synchronize the index with the content of the cache

### The goal of this notebook is to give you the keys to manage the content of your cache by yourself (and potentially setup automatic cleaning at the end of your scripts)
### See the documentation on the tools to manage the cache for more details:
https://climaf.readthedocs.io/en/master/functions_results_cache.html

#  

## First, import climaf

In [1]:
from climaf.api import *

('LC debug :', False)
CliMAF install => /ciclad-home/jservon/Evaluation/CliMAF/climaf_installs/climaf_1.2.12
python => /prodigfs/ipslfs/dods/jservon/miniconda/envs/cesmep_env/bin/python
---
Required softwares to run CliMAF => you are using the following versions/installations:
ncl 6.3.0 => /opt/ncl-6.3.0/bin/ncl
cdo 1.9.6 => /opt/nco/1.9/bin/cdo
nco (ncks) 4.5.2 => /opt/nco-4.5.2/bin/ncks
ncdump 4.4.1.1 => /prodigfs/ipslfs/dods/jservon/miniconda/envs/cesmep_env/bin/ncdump
---


CliMAF version = 1.2.12
Cache directory set to : /data/jservon/climafcache (use $CLIMAF_CACHE if set) 
Cache directory for remote data set to : /data/jservon/climafcache/remote_data (use $CLIMAF_REMOTE_CACHE if set) 
Available macros read from ~/.climaf.macros are : []


### And set verbosity ('critical' -> minimum ; 'debug' -> maximum)

In [2]:
clog('critical') # min verbosity = critical < warning < info < debug = max verbosity

### ... and dont' forget to open the documentation in case you have questions.
### http://climaf.readthedocs.org/
### -> Use the "Quick search" space to search for what you are interested in, it is really powerfull!
###  

# 1. Start with a brand new cache: totally erase the content of the cache

### !! Be careful when you do this!! it will completely remove all the results of your cache!
### !! Consider using a 'test' cache to do this (simply add 'test' at the end of your CLIMAF_CACHE variable, and re-start)

In [3]:
# -- Check which cache your are using
from climaf.cache import cachedirs
cachedirs

['/data/jservon/climafcache']

## If you already have a cache, you can see an index file at its root path:

In [4]:
cachedir = cachedirs[0]
!cat $cachedir/index

(dp0
S"plot(minus(regridn(ccdo(ccdo(ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180'),regridn(ccdo(ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180')))"
p1
S'/data/jservon/climafcache/36dc7/12dea/f1e02/5fb40/ebfcb/12074/1e7e4/0d6ba/ca5af/e5493/c2ce7/d.png'
p2
s.

### clist will retrieve the list of the CRS of the existing results from the index file $CLIMAF_CACHE/index

In [5]:
clist()

Content of CliMAF cache


["plot(minus(regridn(ccdo(ccdo(ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180'),regridn(ccdo(ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180')))"]

## For demo purposes, we will start with a clean cache.
## Use craz() to fully erase your cache and restart with a clean one
https://climaf.readthedocs.io/en/master/functions_results_cache.html?highlight=craz#craz-reset-cache

In [9]:
craz(force=True)

## Now check the content of your cache with clist()
### clist will retrieve the list of the CRS of the existing results from the index file $CLIMAF_CACHE/index

In [10]:
clist()

Content of CliMAF cache


[]

### Because we've just cleaned it, your cache is now totally empty.
#  
## 2. How CliMAF stores its results and knows that those results exist (and are linked with sequences of operations)

### As an example, we will extract a dataset from the CMIP5 archive and do some pretreatments to add results to the cache

In [11]:
# -- Let's get some data as a start
dat_cmip5 = ds(project='CMIP5',
               model='CNRM-CM5',
               variable='tos',
               experiment='historical',
               period='1980-2000',
               frequency='monthly',
               simulation='r1i1p1',
               version='latest'
               )
summary(dat_cmip5)

/bdd/CMIP5/output/CNRM-CERFACS/CNRM-CM5/historical/mon/ocean/Omon/r1i1p1/latest/tos/tos_Omon_CNRM-CM5_historical_r1i1p1_198001-198912.nc
/bdd/CMIP5/output/CNRM-CERFACS/CNRM-CM5/historical/mon/ocean/Omon/r1i1p1/latest/tos/tos_Omon_CNRM-CM5_historical_r1i1p1_199001-199912.nc
/bdd/CMIP5/output/CNRM-CERFACS/CNRM-CM5/historical/mon/ocean/Omon/r1i1p1/latest/tos/tos_Omon_CNRM-CM5_historical_r1i1p1_200001-200512.nc


{'domain': 'global',
 'experiment': 'historical',
 'frequency': 'monthly',
 'model': 'CNRM-CM5',
 'period': 1980-2000,
 'project': 'CMIP5',
 'realization': 'r1i1p1',
 'realm': '*',
 'root': '/bdd',
 'simulation': 'r1i1p1',
 'table': '*',
 'variable': 'tos',
 'version': 'latest'}

## At this stage, dat_cmip5 is only a python object. CliMAF does not load data in memory.

In [12]:
dat_cmip5.crs

"ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest')"

## If I want to get the result of this data request, I use cfile() to return the netcdf file:
https://climaf.readthedocs.io/en/master/functions_results_numeric.html?highlight=cfile#cfile-get-the-file-value-of-a-climaf-object
## CliMAF automatically provides a unique name to the output based on:
 - the root path of the cache ($CLIMAF_CACHE, or the variable cachedirs)
 - a hash of the CRS

In [13]:
cfile(dat_cmip5)

'/data/jservon/climafcache/89f3a/1535f/b106c/37247/4f837/36150/4ecdb/20086/2a547/68ba7/cd864/2.nc'

## When using cfile on your CliMAF object, CliMAF stores the CRS of the new results in the cache in the memory of the python session

In [14]:
clist()

Content of CliMAF cache


["select(ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest'))"]

## If you reinvoke cfile on the same object, it will:
- scan this list
- see that this sequence of operations has already been executed
- and directly return the file associated with it

In [15]:
cfile(dat_cmip5)

'/data/jservon/climafcache/89f3a/1535f/b106c/37247/4f837/36150/4ecdb/20086/2a547/68ba7/cd864/2.nc'

## Let's add some more stuff in the cache

In [16]:
ref = ds(project='ref_climatos', variable='tos', product='WOA13-v2', clim_period='195501-201212')
summary(ref)

/data/jservon/Evaluation/ReferenceDatasets/climatos/ocn/mo/tos/WOA13-v2/ac/tos_Omon_WOA13-v2_observations_195501-201212-clim.nc


{'clim_period': '195501-201212',
 'clim_period_begin': '*',
 'clim_period_end': '*',
 'domain': 'global',
 'frequency': 'annual_cycle',
 'obs_type': '*',
 'period': fx,
 'product': 'WOA13-v2',
 'project': 'ref_climatos',
 'simulation': 'refproduct',
 'table': '*',
 'variable': 'tos'}

In [18]:
# -- Compute a DJF climatology
djf_dat = clim_average(dat_cmip5, 'DJF')
djf_ref = clim_average(ref, 'DJF')

# -- Regrid ref and simulation on the same regular grid
rgrd_dat = regridn(djf_dat, cdogrid='r360x180')
rgrd_ref = regridn(djf_ref, cdogrid='r360x180')

# -- And compute the difference
bias = minus(rgrd_dat, rgrd_ref)

# -- Make a simple plot
myplot = plot(bias)

# -- And execute the final object
cfile(myplot)

'/data/jservon/climafcache/36dc7/12dea/f1e02/5fb40/ebfcb/12074/1e7e4/0d6ba/ca5af/e5493/c2ce7/d.png'

## When you cfile(myplot), CliMAF will:
- analyse the 'tree of operations' that lead to the result
- check if it has the operands of the tree
- use the ones it has
- and compute the ones missing

## The cache now have more results, corresponding to intermediate results leading to the final plot:

In [19]:
clist()

Content of CliMAF cache


["ccdo(ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest'),operator='ymonavg')",
 "select(ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest'))",
 "minus(regridn(ccdo(ccdo(ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180'),regridn(ccdo(ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180'))",
 "ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'),operator='ymonavg')",
 "ccdo(ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12')",
 "select(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'))",
 "plot(minus

## 3. Remove one specific result: cdrop
https://climaf.readthedocs.io/en/master/functions_results_cache.html?highlight=cdrop#cdrop-erase-a-result-s-file

### Let's say that we want to remove the regridded climatologies only. We use cdrop to do this:

In [20]:
cdrop(rgrd_dat)
cdrop(rgrd_ref)

True

### You can confirm with clist() that the files are removed:

In [21]:
clist()

Content of CliMAF cache


["ccdo(ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest'),operator='ymonavg')",
 "select(ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest'))",
 "minus(regridn(ccdo(ccdo(ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180'),regridn(ccdo(ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180'))",
 "ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'),operator='ymonavg')",
 "ccdo(ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12')",
 "select(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'))",
 "plot(minus

## 4. Remove results using conditions (pattern, age, size): crm
https://climaf.readthedocs.io/en/master/functions_results_cache.html?highlight=cdrop#crm-remove-some-files-from-cache

### Now, we would like to clean all the results associated with the CNRM-CM5 simulations
### We use crm(pattern='CNRM-CM5') to remove all the results containing 'CNRM-CM5' in their CRS:

In [22]:
crm(pattern='CNRM-CM5')

Removed files:


["ccdo(ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest'),operator='ymonavg')",
 "minus(regridn(ccdo(ccdo(ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180'),regridn(ccdo(ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180'))",
 "plot(minus(regridn(ccdo(ccdo(ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180'),regridn(ccdo(ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180')))",
 "select(ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest'))",

### Now you should only have the reference, and its annual cycle and DJF climatology

In [23]:
clist()

Content of CliMAF cache


["ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'),operator='ymonavg')",
 "ccdo(ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12')",
 "select(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'))"]

### We can also use crm() to remove files on their age. For instance, let's remove all the results that haven't been touched for more than 20 days:

In [24]:
crm(age='+20')

#### (We've started with a new cache... so the demo might not be very impressive... it won't actually return anything)

### Or remove files on their sizes; remove all the files bigger than 2M

In [25]:
crm(size='2M')

Removed files:


["select(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'))",
 "ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'),operator='ymonavg')"]

### Check the sizes of the elements in the cache with cdu()

In [26]:
cdu()

   1.0K : total
   1.0K : ccdo(ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12')


### => you can combine those conditions:

In [27]:
crm(pattern='CNRM-CM5', age='+20', size='3M')

## 5. cprotect: remove everything but one result (a list of results) you want to keep!

### Let's start by putting some results back in the cache:

In [28]:
# -- And execute the final object
cfile(myplot)

'/data/jservon/climafcache/36dc7/12dea/f1e02/5fb40/ebfcb/12074/1e7e4/0d6ba/ca5af/e5493/c2ce7/d.png'

In [30]:
clist()

Content of CliMAF cache


["ccdo(ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest'),operator='ymonavg')",
 "regridn(ccdo(ccdo(ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180')",
 "minus(regridn(ccdo(ccdo(ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180'),regridn(ccdo(ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180'))",
 "ccdo(ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12')",
 "plot(minus(regridn(ccdo(ccdo(ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest'),operator='ymonavg

### We would like to save only the final result => the plot
### => We will protect 'myplot' in the cache with cprotect()

In [31]:
cprotect(myplot)

In [32]:
clist()

Content of CliMAF cache


["ccdo(ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest'),operator='ymonavg')",
 "regridn(ccdo(ccdo(ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180')",
 "minus(regridn(ccdo(ccdo(ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180'),regridn(ccdo(ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180'))",
 "ccdo(ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12')",
 "plot(minus(regridn(ccdo(ccdo(ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest'),operator='ymonavg

### Now clean everything but the protected results in the cache with craz()

In [33]:
craz()

### And check what we have in the cache now:

In [34]:
clist()

Content of CliMAF cache


["plot(minus(regridn(ccdo(ccdo(ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180'),regridn(ccdo(ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180')))"]

### Big advantage: if we cfile(myplot), we have the result rightaway without having to recompute the intermediate results!
### => save considerable space and still take advantage of the smart cache that finds existing results

### We will make a list of the CRS of the results we want to keep:
### => djf_dat, djf_ref, and myplot

In [35]:
keep_results = []
keep_results.append(djf_dat)
keep_results.append(djf_ref)
keep_results.append(myplot)

In [36]:
keep_results

[ccdo(ccdo(ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),
 ccdo(ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),
 plot(minus(regridn(ccdo(ccdo(ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180'),regridn(ccdo(ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180')))]

### Make a loop on the elements: do cfile on the crs, and cprotect; then craz()

In [37]:
for elt in keep_results:
    cfile(elt)
    cprotect(elt)
craz()
clist()

Content of CliMAF cache


["ccdo(ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12')",
 "plot(minus(regridn(ccdo(ccdo(ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180'),regridn(ccdo(ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180')))",
 "ccdo(ccdo(ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12')"]

### Erase a protected results: cdrop(crs, force=True)

In [38]:
cdrop(myplot, force=True)
clist()

Content of CliMAF cache


["ccdo(ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%195501-201212%*%*%*%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12')",
 "ccdo(ccdo(ds('CMIP5%r1i1p1%tos%1980-2000%global%/bdd%CNRM-CM5%*%historical%r1i1p1%monthly%*%latest'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12')"]

### Erase the whole cache, even protected results: craz(force=True)

In [39]:
craz(force=True)
clist()

Content of CliMAF cache


[]

### This way you only keep the results you really need, if your intermediate results take too much space or inodes!

## 6. Synchronize the file $CLIMAF_CACHE/index with the actual content of the cache: csync(True)
### CliMAF reads the content of the cache at the beginning of the session, and then stores the CRS of the new results in memory (and not in the index file during the session)
### This can have drawbacks when:
- you have multiple processes working on the same cache => typically when you use parallel processes
- some results of the cache have been manually removed
### You then need to synchronize your index file with the actual content of the cache. To do this, we use csync():

In [50]:
# -- Check the content of the index file:
!cat $cachedir/index

(dp0
S"ccdo(ccdo(ds('CMIP5%r1i1p1%tos%1980-2000%global%/prodigfs/project%CNRM-CM5%*%historical%r1i1p1%monthly%*%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12')"
p1
S'/prodigfs/ipslfs/dods/jservon/climafcache_test/00926/2109c/c4299/c4033/34be2/09559/f88d8/0fa7a/372f5/b39b7/95e7f/2.nc'
p2
sS"plot(minus(regridn(ccdo(ccdo(ds('CMIP5%r1i1p1%tos%1980-2000%global%/prodigfs/project%CNRM-CM5%*%historical%r1i1p1%monthly%*%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180'),regridn(ccdo(ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180')))"
p3
S'/prodigfs/ipslfs/dods/jservon/climafcache_test/29523/272b5/8548f/c1823/dd39f/7bfcf/14d49/5b97e/97e1d/0f43d/8ad0c/e.png'
p4
sS"ccdo(ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12')"
p5
S'/prodigfs/ipslfs/dods/jservon/climafcache

In [51]:
# -- Synchronize
csync(True)

In [52]:
# -- And check the index file after
!cat $cachedir/index

(dp0
S"ccdo(ccdo(ds('CMIP5%r1i1p1%tos%1980-2000%global%/prodigfs/project%CNRM-CM5%*%historical%r1i1p1%monthly%*%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12')"
p1
S'/prodigfs/ipslfs/dods/jservon/climafcache_test/00926/2109c/c4299/c4033/34be2/09559/f88d8/0fa7a/372f5/b39b7/95e7f/2.nc'
p2
sS"plot(minus(regridn(ccdo(ccdo(ds('CMIP5%r1i1p1%tos%1980-2000%global%/prodigfs/project%CNRM-CM5%*%historical%r1i1p1%monthly%*%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180'),regridn(ccdo(ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12'),cdogrid='r360x180')))"
p3
S'/prodigfs/ipslfs/dods/jservon/climafcache_test/29523/272b5/8548f/c1823/dd39f/7bfcf/14d49/5b97e/97e1d/0f43d/8ad0c/e.png'
p4
sS"ccdo(ccdo(ds('ref_climatos%refproduct%tos%fx%global%annual_cycle%WOA13-v2%*'),operator='ymonavg'),operator='timmean -seltimestep,1,2,12')"
p5
S'/prodigfs/ipslfs/dods/jservon/climafcache

## This was how to use the CliMAF cache!
##  
##  
## -> go back to the other examples to see more on data access, how to work with ensembles, build an html page...
##  